In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

train=pd.read_csv("train_bsd.csv")
test=pd.read_csv("test_bsd.csv")

print(train.shape)
print(test.shape)

datasets=[train,test]

submit=pd.read_csv("sampleSubmission.csv")
print(submit.shape)
submit.head()

for dataset in datasets:
    dataset["datetime"]=pd.to_datetime(dataset["datetime"])

(10886, 12)
(6493, 9)
(6493, 2)


In [3]:
for dataset in datasets:
    dataset["datetime-year"]=dataset["datetime"].dt.year
    dataset["datetime-hour"]=dataset["datetime"].dt.hour

for dataset in datasets:
    dataset["datetime-dayofweek"]=dataset["datetime"].dt.day_name()
    dataset[["datetime-dayofweek"]]

for dataset in datasets:
    dataset["datetime-dayofweek_Sun"]=dataset["datetime-dayofweek"]=="Sunday"
    dataset["datetime-dayofweek_Mon"]=dataset["datetime-dayofweek"]=="Monday"
    dataset["datetime-dayofweek_Tue"]=dataset["datetime-dayofweek"]=="Tuesday"
    dataset["datetime-dayofweek_Wed"]=dataset["datetime-dayofweek"]=="Wednesday"
    dataset["datetime-dayofweek_Thu"]=dataset["datetime-dayofweek"]=="Thursday"
    dataset["datetime-dayofweek_Fri"]=dataset["datetime-dayofweek"]=="Friday"
    dataset["datetime-dayofweek_Sat"]=dataset["datetime-dayofweek"]=="Saturday"

# variables & model algorithm

In [5]:
feature_names=["season","holiday","workingday","weather","temp","atemp","humidity","windspeed","datetime-year","datetime-hour","datetime-dayofweek_Sun","datetime-dayofweek_Mon","datetime-dayofweek_Tue","datetime-dayofweek_Wed","datetime-dayofweek_Thu","datetime-dayofweek_Fri","datetime-dayofweek_Sat"]
feature_names

['season',
 'holiday',
 'workingday',
 'weather',
 'temp',
 'atemp',
 'humidity',
 'windspeed',
 'datetime-year',
 'datetime-hour',
 'datetime-dayofweek_Sun',
 'datetime-dayofweek_Mon',
 'datetime-dayofweek_Tue',
 'datetime-dayofweek_Wed',
 'datetime-dayofweek_Thu',
 'datetime-dayofweek_Fri',
 'datetime-dayofweek_Sat']

In [6]:
x=train[feature_names]
y=train["count"]
x_test=test[feature_names]

In [7]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(random_state=37)
model

RandomForestRegressor(random_state=37)

# Model Validation
1. 검증하는 이유
* Kaggle에 매번 올리기가 번거로움
* 회사에서 실무 데이터를 다룰 때는 Kaggle로 검증 불가능
2. Data 분석 방식 (3단계)
* 가설 수립
* 검증 &rarr; excel에서는 pivot_table / python에서는 pd.pivot_table 을 이요하여 검증하는 것이 불가능
* 예측
3. 검증 방법
* Hold-out Validation
* Cross Validation

## 1. Hold-out Validation
1. "test"는 미활용
2. "train" data 전체로 fit 하지 않고 일부분으로 fit 시행 (약 80% 이용)
* data를 8:2로 나눈 뒤, 큰 조각을 train data로, 작은 조각을 test data로 가정
* 8000개로 학습(fit)한 뒤 2000개를 예측(predict) &rarr; y(predict) 도출
* 이 y(predict)을 train data의 원래 2000개 값과 비교
* &rarr; 정답이 많이 겹칠수록 model 성능이 좋은 것

## 2. Cross Validation
1. "train" data를 1/n로 나눔 (예: 5조각 (cv=5))
2. 한 조각을 제외한 나머지로 fit 하고 한조각을 predict (n회 시행)
* data를 5조각을 쪼개고, 조각을 뺀 나머지 (=train)로 학습하고, 제외했던 한 조각(=valid)를 예측 (총 5회 시행)
* B+C+D+E &rarr; A 예측
* A+C+D+E &rarr; B 예측
* A+B+D+E &rarr; C 예측
* A+B+C+E &rarr; D 예측
* A+B+C+D &rarr; E 예측
* 도출된 y(predict) 값을 train의 원래 y(actual)값과 비교

<img src="https://velog.velcdn.com/images/dgh0001/post/c594ed0a-9c06-4ade-bfcc-46194b7c3b58/image.png"/>

* 실행속도가 중요하다면 &rarr; Hold-out Validation 사용
* rate-determining step &rarr; "fit" process (fit 횟수와 속도와 비례)

* 정확도가 중요하다면 &rarr; Cross Validation 사용
* Hold-out validation: Kaggle score와 잘 일치하지 않을 수도 있음
* Cross Validation: Kaggle score와 일치할 가능성이 높음

&rarr; Bike Sharing Demand &rarr; Cross Validation으로 검증
* BSD에서는 20 cv로 시행

# Evaluate

In [10]:
from sklearn.model_selection import cross_val_predict
y_predict=cross_val_predict(model,x,y,cv=20)
print(y_predict.shape)
y_predict

(10886,)


array([ 26.69,  23.53,  24.82, ..., 173.26, 171.74, 120.63])

### y_predict와 y를 일일히 비교하기 곤란
### 따라서 Evaluation Metrics(MAE, MSE, RMSE) 사용 검증